In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40

inference_data:
    split: '?'

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?' 
    num_workers: '?' 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0     
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-abs_area_estimation_error'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

## Set the high level name
group: "WMH_AllHospitals_Benchmark"

## Set the name of the dataset so we can automatically load the best models.
evaluate_dataset: "WMH"

# subgroup: "Base"
# subgroup: "Sweep_Threshold"
subgroup: "Sweep_Temperature"
# subgroup: "Optimal_RAVE_Threshold"
# subgroup: "Optimal_RAVE_Temperature"

dataloader:
    batch_size: 2

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# For inference experiments, we don't add the date by default.
add_date = True 
# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg,
    add_date=add_date
)

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
len(inf_cfgs)

16

In [8]:
# Get the first quarter of the configs.
# inf_cfgs = inf_cfgs[:len(inf_cfgs)//4]
# # Get the next quarter of the configs.
# inf_cfgs = inf_cfgs[len(inf_cfgs)//4:len(inf_cfgs)//2]
# # Get the next quarter of the configs.
# inf_cfgs = inf_cfgs[len(inf_cfgs)//2:3*len(inf_cfgs)//4]
# # Get the last quarter of the configs.
inf_cfgs = inf_cfgs[3*len(inf_cfgs)//4:]

## Running Jobs

In [9]:
from ese.analysis.run_inference import get_cal_stats

In [10]:
# from ese.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=inf_cfgs[0], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     show_examples=True,
#     gpu='2',
# )

In [11]:
from ese.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    group="inference",
    base_cfg=updated_base_cfg,
    add_date=add_date,
    exp_cfg=experiment_cfg,
    config_list=inf_cfgs,
    job_func=get_cal_stats,
    available_gpus=['0', '1', '2', '3'],
    # available_gpus=['2', '3', '4', '5', '6', '7']
)

Submitting job 1/4:
--> Launched job-id: 305363 on gpu: 2.
Submitting job 2/4:
--> Launched job-id: 305676 on gpu: 3.
Submitting job 3/4:
--> Launched job-id: 306034 on gpu: 4.
Submitting job 4/4:
--> Launched job-id: 306909 on gpu: 5.
